In [ ]:
#          _____                _____                    _____                    _____                    _____                    _____          
#         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
#        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
#       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
#      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
#     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
#    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
#    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
#  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
# /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
#/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
#\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
# \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
#  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
#   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
#    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
#     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
#      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
#       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
#        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
#         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/    

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Pacman Dependancies

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Create functions for creating video for replay

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Pacman!

In [ ]:

import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

In [ ]:
env = wrap_env(gym.make("MsPacman-v0"))
s = env.reset()

Function for converting code to grayscale (110,90) shape. Credit to Trevor McInroe

In [ ]:
def phi(frame):
    """Expects type batch of tensors (already normed)"""
    # (1) Grayscale
    frame = tf.image.rgb_to_grayscale(frame)
    
    # (2) Downsample
    frame = tf.image.resize(frame, size=[110, 90])
    
    # (3) Crop only the relevant portion
    return frame[:, :90,:]

Display sample image in grayscale

In [ ]:
example_state = phi(tf.convert_to_tensor([s / 255.]))
plt.imshow(tf.squeeze(example_state[0]), cmap='gray')
env.close()

In [ ]:
action_size = env.action_space.n

Set number of episodes to 501 to allow for 500 iterations. Set batch size to 128 and set the neural net training dimensions to (90, 90, 1)

In [ ]:
n_episodes = 501
batch_size=128
input_dims1 = (90,90, 1)

In [ ]:
#check out the pacman action space!
print(env.action_space)

Mount Google drive to save model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Set output directory to Google drive location 

In [ ]:
output_dir = '/content/gdrive/My Drive/msds/462/'

Create directory if it doesn't exist

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Create DQNAgent. This includes creating a neueral net to approximate the Q-value function. Two Conv2d layers are included with 16 and 32 filters followed by a flattening and Dense layer. Output is a linear activation of a output dense layer of size action_size (9). 


In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input

class DQNAgent:
    def __init__(self, action_size,input_dims):
        self.action_size = action_size
        self.input_dims = input_dims
        self.memory = deque(maxlen=10000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Conv2D(input_shape=self.input_dims, filters=16, strides=4,kernel_size=(8,8),activation='relu',padding='same')) # 1st hidden layer; states as input
        model.add(Conv2D(filters=32, strides=2,kernel_size=(4,4),activation='relu',padding='same'))
        model.add(Flatten())
        model.add(Dense(256,activation='relu'))
        model.add(Dense(self.action_size, activation='linear')) # 9 actions, so 9 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, 
                            reward, next_state, done)) # list of previous experiences, enabling re-training later

    def train(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + 
                          self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)
    
    def save(self, name):
        self.model.save_weights(name)

    def load(self, name):
        self.model.load_weights(name)

Initialize the agent 

In [ ]:
agent = DQNAgent(action_size, input_dims1)

In [ ]:
scores = []

In [ ]:
env = wrap_env(gym.make("MsPacman-v0"))

In [ ]:


for e in range(1, n_episodes): # iterate over episodes of gameplay 
    state = env.reset()
    state = phi(tf.convert_to_tensor([state/255])) # reset state at start of each new episode of the game
    done = False
    score = 0
    while not done: 
        #env.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        score = score + reward
        next_state = phi(tf.convert_to_tensor([next_state/255]))
        #next_state = np.reshape(next_state, [90, 90])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # if episode ends: 
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes-1, score, agent.epsilon))
        scores.append(score)
    if len(agent.memory) > batch_size:
        agent.train(batch_size) # train the agent by replaying the experiences of the episode
    if e % 20 == 0:
        agent.save(output_dir + "weights_latest_train_3" 
                   + '{:04d}'.format(e) + ".hdf5")
env.close()

In [ ]:
#agent = DQNAgent( action_size,  input_dims1)
agent.load(output_dir + 'weights_latest_train_30500.hdf5')

In [ ]:
env = wrap_env(gym.make("MsPacman-v0"))

state = env.reset()
state = phi(tf.convert_to_tensor([state/255])) # reset state at start of each new episode of the game
done = False
score = 0
# time represents a frame of the episode; goal is to keep pole upright as long as possible
while not done: 
    env.render()
    action = agent.act(state) # action is to move in some direction
    next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 
    score = score + reward       
    next_state = next_state
    next_state = phi(tf.convert_to_tensor([next_state/255]))

    state = next_state # set "current state" for upcoming iteration to the current next state        

env.close()
show_video()